In [25]:
import streamlit as st
import pandas as pd
import plotly.express as px

In [26]:
df = pd.read_csv("data/acidentes2022.csv", sep=';')

In [27]:
colunas_drop = ['vitimasfatais', 'auto', 'moto', 'ciclom', 'ciclista', 'pedestre', 'onibus', 'caminhao', 'viatura', 'outros', 'vitimas']
for coluna in colunas_drop:
    df[coluna] = df[coluna].str.split(',').str[0].astype(int)

In [28]:

index_null_tempo = list(df[df['tempo_clima'].isnull()].index)

for index in index_null_tempo:
    date_tempo = df.loc[index, 'data']
    aux_df = df[df['data'] == date_tempo]
    value_mode = aux_df['tempo_clima'].mode()
    
    if len(value_mode) > 0:
        df.loc[index, 'tempo_clima'] = value_mode.iloc[0]

# Preencher valores nulos restantes com "Não especificado"
df['tempo_clima'].fillna("Não especificado", inplace=True)

# Agrupar por 'tempo_clima' e somar 'vitimas'
acidentes_por_clima = df.groupby('tempo_clima')['vitimasfatais'].sum().reset_index()
acidentes_por_clima

tempo_clima  vitimasfatais
0               Bom             19
1           Chuvoso              6
2           Nublado              0
3  Não especificado              0

In [29]:
df.head(5)

Protocolo        data  hora    natureza    situacao         bairro  \
0  202200001,0  2022-01-01   NaN  COM VÍTIMA  FINALIZADA         JORDÃO   
1  202200007,0  2022-01-01   NaN  SEM VÍTIMA  FINALIZADA    IMBIRIBEIRA   
2  202200025,0  2022-01-01   NaN  COM VÍTIMA   CANCELADA        CAXANGÁ   
3  202200029,0  2022-01-01   NaN  COM VÍTIMA   CANCELADA  JOANA BEZERRA   
4  202200082,0  2022-01-02   NaN  COM VÍTIMA  FINALIZADA        AFLITOS   

                            endereco numero detalhe_endereco_acidente  \
0                     AV MARIA IRENE  130,0                       NaN   
1  AV MARECHAL MASCARENHAS DE MORAES    NaN                       NaN   
2                         AV CAXANGA    NaN                       NaN   
3   AV GOVERNADOR AGAMENON MAGALHAES    NaN                       NaN   
4             RUA MANUEL DE CARVALHO  432,0       RUA CARNEIRO VILELA   

                                         complemento  ...      sinalizacao  \
0                                                NaN  ...    Não existente   
1                         LADO OPOSTO AO NÚMERO 4728  ...  Perfeito estado   
2  PROX AO CRUZAMENTO  DE SAN MARTIN E/F A IGREJA...  ...              NaN   
3  ENFRENTE AO FORUM DE JOANA BEZERRA SENTIDO BOA...  ...              NaN   
4                           PRÓX AO CAMPO DO NAUTICO  ...  Perfeito estado   

  condicao_via  conservacao_via     ponto_controle  situacao_placa  \
0         Seca  Perfeito estado         Não existe   Não há placas   
1         Seca  Perfeito estado  Faixa de pedestre   Não há placas   
2          NaN              NaN                NaN             NaN   
3          NaN              NaN                NaN             NaN   
4         Seca  Perfeito estado                NaN   Placas "Pare"   

   velocidade_max_via  mao_direcao      divisao_via1  divisao_via2  \
0                 NaN        Dupla        Não existe           NaN   
1                 NaN        Dupla  Canteiro central           NaN   
2                 NaN          NaN               NaN           NaN   
3                 NaN          NaN               NaN           NaN   
4                 NaN        Única    Faixa contínua           NaN   

   divisao_via3  
0           NaN  
1           NaN  
2           NaN  
3           NaN  
4           NaN  

[5 rows x 38 columns]

In [30]:
df.columns

Index(['Protocolo', 'data', 'hora', 'natureza', 'situacao', 'bairro',
       'endereco', 'numero', 'detalhe_endereco_acidente', 'complemento',
       'bairro_cruzamento', 'num_semaforo', 'sentido_via', 'tipo', 'auto',
       'moto', 'ciclom', 'ciclista', 'pedestre', 'onibus', 'caminhao',
       'viatura', 'outros', 'vitimas', 'vitimasfatais', 'acidente_verificado',
       'tempo_clima', 'situacao_semaforo', 'sinalizacao', 'condicao_via',
       'conservacao_via', 'ponto_controle', 'situacao_placa',
       'velocidade_max_via', 'mao_direcao', 'divisao_via1', 'divisao_via2',
       'divisao_via3'],
      dtype='object')

In [31]:
df = df.drop(['numero', 'detalhe_endereco_acidente', 'complemento', 'bairro_cruzamento'], axis=1)

In [32]:
df.head(5)

Protocolo        data  hora    natureza    situacao         bairro  \
0  202200001,0  2022-01-01   NaN  COM VÍTIMA  FINALIZADA         JORDÃO   
1  202200007,0  2022-01-01   NaN  SEM VÍTIMA  FINALIZADA    IMBIRIBEIRA   
2  202200025,0  2022-01-01   NaN  COM VÍTIMA   CANCELADA        CAXANGÁ   
3  202200029,0  2022-01-01   NaN  COM VÍTIMA   CANCELADA  JOANA BEZERRA   
4  202200082,0  2022-01-02   NaN  COM VÍTIMA  FINALIZADA        AFLITOS   

                            endereco num_semaforo sentido_via  \
0                     AV MARIA IRENE          NaN    SUBURBIO   
1  AV MARECHAL MASCARENHAS DE MORAES        471,0      CIDADE   
2                         AV CAXANGA          NaN         NaN   
3   AV GOVERNADOR AGAMENON MAGALHAES          NaN         NaN   
4             RUA MANUEL DE CARVALHO          NaN    SUBURBIO   

               tipo  ...      sinalizacao  condicao_via  conservacao_via  \
0   COLISÃO FRONTAL  ...    Não existente          Seca  Perfeito estado   
1  COLISÃO TRASEIRA  ...  Perfeito estado          Seca  Perfeito estado   
2  COLISÃO TRASEIRA  ...              NaN           NaN              NaN   
3   COLISÃO LATERAL  ...              NaN           NaN              NaN   
4   COLISÃO LATERAL  ...  Perfeito estado          Seca  Perfeito estado   

      ponto_controle  situacao_placa  velocidade_max_via  mao_direcao  \
0         Não existe   Não há placas                 NaN        Dupla   
1  Faixa de pedestre   Não há placas                 NaN        Dupla   
2                NaN             NaN                 NaN          NaN   
3                NaN             NaN                 NaN          NaN   
4                NaN   Placas "Pare"                 NaN        Única   

       divisao_via1  divisao_via2  divisao_via3  
0        Não existe           NaN           NaN  
1  Canteiro central           NaN           NaN  
2               NaN           NaN           NaN  
3               NaN           NaN           NaN  
4    Faixa contínua           NaN           NaN  

[5 rows x 34 columns]

In [33]:
#df.unique['bairro']

In [34]:
import json


local_path = 'data/bvisualizacao_fcbairro.geojson'

with open(local_path, 'r', encoding='utf-8') as file:
    state_geo = json.load(file)

print(type(state_geo))

FileNotFoundError: [Errno 2] No such file or directory: 'data/bvisualizacao_fcbairro.geojson'

In [ ]:
print((state_geo['features'][0]['properties']['EBAIRRNOME']))
print((df['bairro'][1]))

CURADO
IMBIRIBEIRA


In [ ]:
acidentes_por_bairro = df.groupby('bairro')['vitimas'].count()
acidentes_por_bairro = acidentes_por_bairro.reset_index()
acidentes_por_bairro

bairro  vitimas
0               AFLITOS        5
1              AFOGADOS       86
2         ALTO DO MANDU        5
3   ALTO JOSÉ BONIFÁCIO        4
4    ALTO JOSÉ DO PINHO        1
..                  ...      ...
86                 TOTÓ        9
87        VASCO DA GAMA       17
88               VÁRZEA       57
89                ZUMBI        3
90            ÁGUA FRIA       31

[91 rows x 2 columns]

In [ ]:
import plotly.express as px

fig = px.choropleth(acidentes_por_bairro, geojson=state_geo, color="vitimas",
                    locations="bairro", featureidkey="properties.EBAIRRNOME",
                    projection="mercator",
                    labels={'População':'Population'},
                   )
fig.update_geos(fitbounds="geojson", visible=True)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
import folium
map = folium.Map(
    location=[-14.2350,-51.9253],
    tiles='cartodbpositron', #cartodbpositron, Stamen Terrain, Stamen Toner
    zoom_start=4,
)

#Colorindo o mapa
folium.Choropleth(
    geo_data=state_geo,
    line_opacity=0.5,
    line_color='black',
    highlight=True,
    legend_name='Population',
).add_to(map)

#Exibição do mapa
map

In [ ]:
vitimas = df.groupby('natureza')['vitimas'].sum()
vitimas = vitimas.sort_values(ascending=False)
vitimas - vitimas.reset_index(name='total')
vitimas

natureza
COM VÍTIMA      1,02,01,01,01,01,01,01,01,01,01,01,01,02,01,01...
VÍTIMA FATAL    0,00,00,00,00,00,00,00,01,00,01,01,00,00,00,00...
SEM VÍTIMA      0,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00...
Name: vitimas, dtype: object

In [35]:
df = pd.read_csv("data/acidentes2018.csv", sep=';')

In [36]:
df.columns

Index(['DATA', 'hora', 'natureza_acidente', 'situacao', 'bairro', 'endereco',
       'numero', 'detalhe_endereco_acidente', 'complemento',
       'endereco_cruzamento', 'numero_cruzamento', 'referencia_cruzamento',
       'bairro_cruzamento', 'num_semaforo', 'sentido_via', 'tipo', 'descricao',
       'auto', 'moto', 'ciclom', 'ciclista', 'pedestre', 'onibus', 'caminhao',
       'viatura', 'outros', 'vitimas', 'vitimasfatais', 'acidente_verificado',
       'tempo_clima', 'situacao_semaforo', 'sinalizacao', 'condicao_via',
       'conservacao_via', 'ponto_controle', 'situacao_placa',
       'velocidade_max_via', 'mao_direcao', 'divisao_via1', 'divisao_via2',
       'divisao_via3'],
      dtype='object')

In [37]:
df.rename(columns={'DATA': 'data'}, inplace=True)

In [38]:
df.columns

Index(['data', 'hora', 'natureza_acidente', 'situacao', 'bairro', 'endereco',
       'numero', 'detalhe_endereco_acidente', 'complemento',
       'endereco_cruzamento', 'numero_cruzamento', 'referencia_cruzamento',
       'bairro_cruzamento', 'num_semaforo', 'sentido_via', 'tipo', 'descricao',
       'auto', 'moto', 'ciclom', 'ciclista', 'pedestre', 'onibus', 'caminhao',
       'viatura', 'outros', 'vitimas', 'vitimasfatais', 'acidente_verificado',
       'tempo_clima', 'situacao_semaforo', 'sinalizacao', 'condicao_via',
       'conservacao_via', 'ponto_controle', 'situacao_placa',
       'velocidade_max_via', 'mao_direcao', 'divisao_via1', 'divisao_via2',
       'divisao_via3'],
      dtype='object')

In [39]:
df['data'] = pd.to_datetime(df['data'])

In [41]:
df['ano'] = df['data'].dt.year

In [43]:
df['ano'].unique()

array([2018])

In [44]:
df_2016 = pd.read_csv("data/acidentes2016.csv", sep=';')
df_2017 = pd.read_csv("data/acidentes2017.csv", sep=';')
df_2018 = pd.read_csv("data/acidentes2018.csv", sep=';')
df_2019 = pd.read_csv("data/acidentes2019.csv", sep=';')
df_2020 = pd.read_csv("data/acidentes2020.csv", sep=';')
df_2021 = pd.read_csv("data/acidentes2021.csv", sep=';')
df_2022 = pd.read_csv("data/acidentes2022.csv", sep=';')

In [52]:
df_2022.dtypes

Protocolo                     object
data                          object
hora                         float64
natureza                      object
situacao                      object
bairro                        object
endereco                      object
numero                        object
detalhe_endereco_acidente     object
complemento                   object
bairro_cruzamento             object
num_semaforo                  object
sentido_via                   object
tipo                          object
auto                          object
moto                          object
ciclom                        object
ciclista                      object
pedestre                      object
onibus                        object
caminhao                      object
viatura                       object
outros                        object
vitimas                       object
vitimasfatais                 object
acidente_verificado           object
tempo_clima                   object
s